In [1]:
import os
import openai
from dotenv import load_dotenv, find_dotenv
_ = load_dotenv(find_dotenv())
openai.api_key = os.environ['OPENAI_API_KEY']


In [2]:
import json
def get_current_weather(location, unit="fahrenheit"):
    weather_info = {
        "location": location,
        "temperature": "72",
        "unit": unit,
        "forecast": ["sunny", "windy"],
    }
    return json.dumps(weather_info)


In [3]:
functions = [
    {
        "name": "get_current_weather",
        "description": "Get the current weather in a given location",
        "parameters": {
            "type": "object",
            "properties": {
                "location": {"type": "string", "description": "The city and state, e.g. San Francisco, CA"},
                "unit": {"type": "string", "enum": ["celsius", "fahrenheit"]},
            },
            "required": ["location"],
        },
    }
]


In [4]:
messages = [
    {
        "role": "user",
        "content": "What's the weather like in Boston?"
    }
]


In [5]:
response = openai.ChatCompletion.create(

    model="gpt-3.5-turbo",
    messages=messages,
    functions=functions,
)

In [6]:
response_message = response["choices"][0]["message"]
print(response_message["function_call"])


{
  "name": "get_current_weather",
  "arguments": "{\"location\":\"Boston\",\"unit\":\"celsius\"}"
}


In [7]:
import json

args = json.loads(response_message["function_call"]["arguments"])
result = get_current_weather(args["location"], args["unit"])


In [8]:
messages.append(response_message)  # function_call
messages.append({
    "role": "function",
    "name": "get_current_weather",
    "content": result,
})


In [9]:
response = openai.ChatCompletion.create(
    model="gpt-3.5-turbo",
    messages=messages,
)
print(response["choices"][0]["message"]["content"])


The weather in Boston is currently 72°C with sunny and windy conditions.


In [10]:
import os
from dotenv import load_dotenv
load_dotenv()

from langchain.chat_models import ChatOpenAI
model = ChatOpenAI()  # Uses OpenAI under the hood


C:\Users\vikas\AppData\Local\Temp\ipykernel_23568\1098582902.py:6: LangChainDeprecationWarning: The class `ChatOpenAI` was deprecated in LangChain 0.0.10 and will be removed in 1.0. An updated version of the class exists in the :class:`~langchain-openai package and should be used instead. To use it run `pip install -U :class:`~langchain-openai` and import as `from :class:`~langchain_openai import ChatOpenAI``.
  model = ChatOpenAI()  # Uses OpenAI under the hood


## Function Calling Using Langchain

In [11]:
from pydantic import BaseModel, Field

class WeatherSearch(BaseModel):
    """Call this with an airport code to get the weather at that airport"""
    airport_code: str = Field(description="airport code to get weather for")


In [12]:
from langchain.utils.openai_functions import convert_pydantic_to_openai_function
weather_function = convert_pydantic_to_openai_function(WeatherSearch)

C:\Users\vikas\AppData\Local\Temp\ipykernel_23568\2553069172.py:2: LangChainDeprecationWarning: The function `_convert_pydantic_to_openai_function` was deprecated in LangChain 0.1.16 and will be removed in 1.0. Use :meth:`~langchain_core.utils.function_calling.convert_to_openai_function()` instead.
  weather_function = convert_pydantic_to_openai_function(WeatherSearch)


In [13]:
weather_function

{'name': 'WeatherSearch',
 'description': 'Call this with an airport code to get the weather at that airport',
 'parameters': {'properties': {'airport_code': {'description': 'airport code to get weather for',
    'type': 'string'}},
  'required': ['airport_code'],
  'type': 'object'}}

In [14]:
from langchain.chat_models import ChatOpenAI

In [15]:
model = ChatOpenAI()

In [16]:
model.invoke("what is the weather in SF today?", functions=[weather_function])



AIMessage(content='', additional_kwargs={'function_call': {'name': 'WeatherSearch', 'arguments': '{"airport_code":"SFO"}'}}, response_metadata={'token_usage': <OpenAIObject at 0x1b97e5273d0> JSON: {
  "prompt_tokens": 70,
  "completion_tokens": 17,
  "total_tokens": 87,
  "prompt_tokens_details": {
    "cached_tokens": 0,
    "audio_tokens": 0
  },
  "completion_tokens_details": {
    "reasoning_tokens": 0,
    "audio_tokens": 0,
    "accepted_prediction_tokens": 0,
    "rejected_prediction_tokens": 0
  }
}, 'model_name': 'gpt-3.5-turbo', 'system_fingerprint': None, 'finish_reason': 'function_call', 'logprobs': None}, id='run--572c53bb-3bfc-4101-9824-df81f6a7de94-0')

In [17]:
model_with_function = model.bind(functions=[weather_function])

In [18]:
model_with_function.invoke("What is the weather at SFO?")


AIMessage(content='', additional_kwargs={'function_call': {'name': 'WeatherSearch', 'arguments': '{"airport_code":"SFO"}'}}, response_metadata={'token_usage': <OpenAIObject at 0x1b97f56a7a0> JSON: {
  "prompt_tokens": 70,
  "completion_tokens": 17,
  "total_tokens": 87,
  "prompt_tokens_details": {
    "cached_tokens": 0,
    "audio_tokens": 0
  },
  "completion_tokens_details": {
    "reasoning_tokens": 0,
    "audio_tokens": 0,
    "accepted_prediction_tokens": 0,
    "rejected_prediction_tokens": 0
  }
}, 'model_name': 'gpt-3.5-turbo', 'system_fingerprint': None, 'finish_reason': 'function_call', 'logprobs': None}, id='run--d98a60a5-e568-4e2c-a595-6bf88bbd4760-0')

In [19]:
model_with_forced_function = model.bind(
    functions=[weather_function],
    function_call={"name": "WeatherSearch"}
)

model_with_forced_function.invoke("hi!")  # GPT must call your tool


AIMessage(content='', additional_kwargs={'function_call': {'name': 'WeatherSearch', 'arguments': '{"airport_code":"SFO"}'}}, response_metadata={'token_usage': <OpenAIObject at 0x1b97f54bdd0> JSON: {
  "prompt_tokens": 74,
  "completion_tokens": 7,
  "total_tokens": 81,
  "prompt_tokens_details": {
    "cached_tokens": 0,
    "audio_tokens": 0
  },
  "completion_tokens_details": {
    "reasoning_tokens": 0,
    "audio_tokens": 0,
    "accepted_prediction_tokens": 0,
    "rejected_prediction_tokens": 0
  }
}, 'model_name': 'gpt-3.5-turbo', 'system_fingerprint': None, 'finish_reason': 'stop', 'logprobs': None}, id='run--8049f680-2df0-4923-a902-b86781fe5676-0')

In [20]:
from langchain.prompts import ChatPromptTemplate

prompt = ChatPromptTemplate.from_messages([
    ("system", "You are a helpful assistant."),
    ("user", "{input}")
])

chain = prompt | model_with_function
chain.invoke({"input": "What is the weather in SF?"})


AIMessage(content='', additional_kwargs={'function_call': {'name': 'WeatherSearch', 'arguments': '{"airport_code":"SFO"}'}}, response_metadata={'token_usage': <OpenAIObject at 0x1b97f588900> JSON: {
  "prompt_tokens": 75,
  "completion_tokens": 17,
  "total_tokens": 92,
  "prompt_tokens_details": {
    "cached_tokens": 0,
    "audio_tokens": 0
  },
  "completion_tokens_details": {
    "reasoning_tokens": 0,
    "audio_tokens": 0,
    "accepted_prediction_tokens": 0,
    "rejected_prediction_tokens": 0
  }
}, 'model_name': 'gpt-3.5-turbo', 'system_fingerprint': None, 'finish_reason': 'function_call', 'logprobs': None}, id='run--54e9c10f-4272-4338-b96a-242f2b3ef4c2-0')

In [21]:
class ArtistSearch(BaseModel):
    """Call this to get the names of songs by a particular artist"""
    artist_name: str = Field(description="name of artist to look up")
    n: int = Field(description="number of results")


In [22]:
functions = [
    convert_pydantic_to_openai_function(WeatherSearch),
    convert_pydantic_to_openai_function(ArtistSearch)
]


In [23]:
model_with_functions = model.bind(functions=functions)


In [24]:
model_with_functions.invoke("What are 3 songs by Taylor Swift?")
model_with_functions.invoke("What is the weather at JFK?")
model_with_functions.invoke("Hello!")


AIMessage(content='Hi there! How can I assist you today?', additional_kwargs={}, response_metadata={'token_usage': <OpenAIObject at 0x1b97f6e07c0> JSON: {
  "prompt_tokens": 111,
  "completion_tokens": 11,
  "total_tokens": 122,
  "prompt_tokens_details": {
    "cached_tokens": 0,
    "audio_tokens": 0
  },
  "completion_tokens_details": {
    "reasoning_tokens": 0,
    "audio_tokens": 0,
    "accepted_prediction_tokens": 0,
    "rejected_prediction_tokens": 0
  }
}, 'model_name': 'gpt-3.5-turbo', 'system_fingerprint': None, 'finish_reason': 'stop', 'logprobs': None}, id='run--e227baa4-7578-4dcf-a7a9-5bd831f97223-0')

In [25]:
print("The End")

The End
